## **CT1-MLOPS COURSE GROUP ASSIGNMENT** ##

> **Use Case ::** Predicting Student Grade

> **Dataset Source ::** Kaggle - https://www.kaggle.com/datasets/rabieelkharoua/students-performance-dataset/data

> **Group No. ::** 12

## **STEP 9 :: USER INTERFACE - STREAMLIT**

In [ ]:
pip install streamlit altair uvicorn pyngrok --no-warn-script-location

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00


In [ ]:
# Importing necessary libraries
from multiprocessing import Process
from threading import Thread
from pyngrok import ngrok
import streamlit as st
import pandas as pd
import numpy as np
from joblib import load
import os

# Google Colab Notbook related imports
from google.colab import userdata

### **VII.  User Interface - Streamlit**
> In this stage we will creating a User interface for user to select the input values and display the predicted value through the best performance model deployed. The UI app will use the FastAPI service hosted at localhost:8501 to post the post the request to model and fetch the value.

> **Streamlit -**
>> Streamlit is a free and open-source framework to rapidly build and share web apps without extensive web development knowledge.

#### **Creating the APP**
> We will be defining the app layout i.e., the fields that are to be displayed to the user over the screen and the type of field to be provided (e.g., dropdown for stratified data or text field etc.) along the set of values acceptable in each field (e.g, range of values for free text or definite values to be displayed in dropdown).

> **Demographic Details**

>>**Age:** The age of the students ranges from 15 to 18 years.

>>**Gender:** Gender of the students, where 0 represents Male and 1 represents Female.

>>**Ethnicity:** The ethnicity of the students, coded as follows:
 >>- 0: Caucasian
 >>- 1: African American
 >>- 2: Asian
 >>- 3: Other

>>**ParentalEducation:** The education level of the parents, coded as follows:
 >>- 0: None
 >>- 1: High School
 >>- 2: Some College
 >>- 3: Bachelor's
 >>- 4: Higher

> **Study Habits**

>> **StudyTimeWeekly:** Weekly study time in hours, ranging from 0 to 20.

>> **Absences:** Number of absences during the school year, ranging from 0 to 30.

>> **Tutoring:** Tutoring status, where 0 indicates No and 1 indicates Yes.

> **Parental Involvement**
>> **ParentalSupport:** The level of parental support, coded as follows:
>> - 0: None
>> - 1: Low
>> - 2: Moderate
>> - 3: High
>> - 4: Very High

>**Extracurricular Activities**
>> **Extracurricular:** Participation in extracurricular activities, where 0 indicates No and 1 indicates Yes.

>> **Sports:** Participation in sports, where 0 indicates No and 1 indicates Yes.

>> **Music:** Participation in music activities, where 0 indicates No and 1 indicates Yes.

>> **Volunteering:** Participation in volunteering, where 0 indicates No and 1 indicates Yes.

>**Academic Performance**
>> **GPA:** Grade Point Average on a scale from 0.0 to 4.0, influenced by study habits, parental involvement, and extracurricular activities.

>**Target Variable:**
>> **GradeClass:** Classification of students' grades based on GPA:
>>- 0: 'A' (GPA >= 3.5)
>>- 1: 'B' (3.0 <= GPA < 3.5)
>>- 2: 'C' (2.5 <= GPA < 3.0)
>>- 3: 'D' (2.0 <= GPA < 2.5)
>>- 4: 'E' (GPA < 2.0)

**Steps Performed -**
1. We create a python file app.py that contains the outline of the webpage that is to be generated via streamlit library. For this we define the page title name, header, parameters that are to be displayed on the page and whose values are to be captured and posted to once entered by user on the web page.
2. For each parameter defined, we provide the type of field (slider, free text box etc.), data type and range of values to be accepted.
3. We then define the payload format that is to be posted when the "Predict" button is clicked.
4. The web application is hosted at a localserver started at endpoint 0.0.0.0:8501 using the uvicorn module. The streamlit module automatically converts the outline defined in app.py to html content with a user friendly interface.
5. When user enters the details and clicks Predict the request is received by the app and posted to the secure API endpoint exposed over public URL.

**Note - Pls ensure the "ngrokPublicURL.txt" file to be fetched and uploaded from Notebook-2 to the local storage of this notebook.**

**Using NGROK::**

- We use ngrok to create a secure endpoint to which an external user can access the web application hosted. Ngrok creates a secure tunnel between the public exposed endpoint and the local server (0.0.0.0:8501) end point on which the service is running.
- Once the enters the details on the page and hits the Predict button,traffic/request hits the public endpoint, ngrok forwards the traffic over the secure channel thereby abstracting the internal working endpoint from outside world.


In [ ]:
pip install scikit-learn==1.3.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 69.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2


In [ ]:
%%writefile app.py

from pyngrok import ngrok
import streamlit as st
import pandas as pd
import numpy as np
from joblib import load
import requests

# Loading the FastAPI public URL from the file to which we had exported the value in previous DeploymentCode File No.2
with open("ngrokPublicURL.txt", "r") as f:
    endPointUrl = f.read().strip()

# Function to define the app_layout
def app_layout():

    st.title('Student Grade Class Predictor')
    st.header('Enter the below details ::')

    # Creating the user input fields
    Age = st.number_input('Age:',
                          min_value=15,
                          max_value=18,
                          value=15)

    Gender = st.radio('Gender:',
                      ['Male', 'Female'],
                      horizontal=True)
    # Mapping Gender to the numeric value 0 and 1 for backend
    Gender = 0 if Gender == "Male" else 1

    Ethnicity = st.selectbox("Ethnicity:",
                             options=['Caucasian', 'African American', 'Asian', 'Other'])
    # Mapping Ethnicity to the numeric value 0 to 3 for backend
    match Ethnicity:
        case 'Caucasian':
            Ethnicity = 0
        case 'African American':
            Ethnicity = 1
        case 'Asian':
            Ethnicity = 2
        case 'Other':
            Ethnicity = 3

    ParentalEducation = st.selectbox("Parental Education:",
                             options=[ 'None', 'High School', 'Some College', 'Bachelors', 'Higher'])
    # Mapping Ethnicity to the numeric value 0 to 3 for backend
    match ParentalEducation:
        case 'None':
            ParentalEducation = 0
        case 'High School':
            ParentalEducation = 1
        case 'Some College':
            ParentalEducation = 2
        case 'Bachelors':
            ParentalEducation = 3
        case 'Higher':
            ParentalEducation = 4

    StudyTimeWeekly = st.slider("Study Time Weekly (hours):",
                                min_value=0.000,
                                max_value=20.000,
                                step=0.001)

    Absences = st.slider("Absences:",
                         min_value=0,
                         max_value=30,
                         step=1)

    Tutoring = st.radio('Tutoring:',
                        ['Yes', 'No'],
                        horizontal=True)
    # Mapping Tutoring to the numeric value 0 and 1 for backend
    Tutoring = 0 if Tutoring == "No" else 1

    ParentalSupport = st.selectbox("Parental Support:",
                                   options=['None', 'Low', 'Moderate', 'High', 'Very High'])
    # Mapping ParentalSupport to the numeric value 0 to 3 for backend
    match ParentalSupport:
        case 'None':
            ParentalSupport = 0
        case 'Low':
            ParentalSupport = 1
        case 'Moderate':
            ParentalSupport = 2
        case 'High':
            ParentalSupport = 3
        case 'Very High':
            ParentalSupport = 4

    Extracurricular = st.radio('Extracurricular:',
                               ['Yes', 'No'],
                               horizontal=True)
    # Mapping Extracurricular to the numeric value 0 and 1 for backend
    Extracurricular = 0 if Extracurricular == "No" else 1

    Sports = st.radio('Sports:',
                      ['Yes', 'No'],
                      horizontal=True)
    # Mapping Sports to the numeric value 0 and 1 for backend
    Sports = 0 if Sports == "No" else 1

    Music = st.radio('Music:',
                     ['Yes', 'No'],
                     horizontal=True)
    # Mapping Music to the numeric value 0 and 1 for backend
    Music = 0 if Music == "No" else 1

    Volunteering = st.radio('Volunteering:',
                            ['Yes', 'No'],
                            horizontal=True)
    # Mapping Music to the numeric value 0 and 1 for backend
    Volunteering = 0 if Volunteering == "No" else 1

    if st.button('Predict Grade'):
        #Constructing the payload to be posted via FastAPI
        payload = {
            "Age": Age,
            "Gender": Gender,
            "Ethnicity": Ethnicity,
            "ParentalEducation": ParentalEducation,
            "StudyTimeWeekly": StudyTimeWeekly,
            "Absences": Absences,
            "Tutoring": Tutoring,
            "ParentalSupport": ParentalSupport,
            "Extracurricular": Extracurricular,
            "Sports": Sports,
            "Music": Music,
            "Volunteering": Volunteering,
        }


        # Make a POST request to the FastAPI server
        gradeClass = requests.post(endPointUrl + "/predict", json=payload)

        # Print the status code
        print(f"Status Code: {gradeClass.status_code}")

        # Print the headers
        print(f"Headers: {gradeClass.headers}")

        # Print the content of the response (the actual prediction)
        print(f"Response Text: {gradeClass.text}")

        # Clean the string and split based on commas (or other delimiters within [])
        gradeClass = gradeClass.text.replace('[', '').replace(']', '')
        st.success(gradeClass)

if __name__=='__main__':
  app_layout()


Writing app.py


#### **Running the UI**
> The UI service shall be hosted on port 9090 in localhost.

In [ ]:
# Storing the ngrok auth-token which will be later used to authorize the web user posting the API request when connecting to the API service hosted at port 8000
ngrok.set_auth_token("2oy6VhkcQYpcuGwVNoaRhKrA5T6_vwBUcQ4iShdyFHoMnCD4")

In [ ]:
# Start the Streamlit server in a separate thread so that the execution of main programme running in this notebook is not interrupted
streamlit_thread = Thread(
    target=lambda: os.system("streamlit run app.py --server.port 8501"), daemon=True
)
streamlit_thread.start()

# Expose the Streamlit app through ngrok
# ngrokPublicURL.txt file to be fetched and uploaded from Notebook-2
streaming_url = ngrok.connect(8501)
print(f"Streamlit public URL: {streaming_url}")

Streamlit public URL: NgrokTunnel: "https://db22-35-227-98-54.ngrok-free.app" -> "http://localhost:8501"
